In [3]:
import os
import json

import tqdm

In [4]:
path = '/Users/vanek/PycharmProjects/tutor-llm/data/math/MATH/test'

In [5]:
category_data = {}


for category in os.listdir(path):
    print(category)
    category_data[category] = []
    for file in os.listdir(os.path.join(path, category)):
        with open(os.path.join(path, category, file), 'r') as f:
            category_data[category].append(json.load(f))

counting_and_probability
intermediate_algebra
number_theory
precalculus
prealgebra
geometry
algebra


In [6]:
sum_len = 0
for c in category_data:
    print(c, len(category_data[c]))
    sum_len += len(category_data[c])
print(sum_len)

counting_and_probability 474
intermediate_algebra 903
number_theory 540
precalculus 546
prealgebra 871
geometry 479
algebra 1187
5000


In [7]:
category_data['precalculus'][-12]

{'problem': 'The set of vectors $\\mathbf{v}$ such that\n\\[\\operatorname{proj}_{\\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix}} \\mathbf{v} = \\begin{pmatrix} 0 \\\\ 0 \\end{pmatrix}\\]lie on a line.  Enter the equation of this line in the form "$y = mx + b$".',
 'level': 'Level 3',
 'type': 'Precalculus',
 'solution': 'Let $\\mathbf{v} = \\begin{pmatrix} x \\\\ y \\end{pmatrix}.$\n\nFrom the formula of a projection,\n\\begin{align*}\n\\operatorname{proj}_{\\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix}} \\mathbf{v} &= \\frac{\\mathbf{v} \\cdot \\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix}}{\\left\\| \\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix} \\right\\|^2} \\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix} \\\\\n&= \\frac{\\begin{pmatrix} x \\\\ y \\end{pmatrix} \\cdot \\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix}}{5} \\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix} \\\\\n&= \\frac{2x + y}{5} \\begin{pmatrix} 2 \\\\ 1 \\end{pmatrix} \\\\\n&= \\begin{pmatrix} 0 \\\\ 0 \\end{pmatrix}.\n\\end{align*}Then\n\\[\\frac{2x + y}{5} = 

In [8]:
from datasets.math_dataset import normalize_final_answer, remove_boxed, last_boxed_only_string

doc = category_data['precalculus'][-12]

normalize_final_answer(remove_boxed(last_boxed_only_string(doc["solution"])))

'-2x'

In [9]:

from datasets.math_dataset import list_fewshot_samples
def nshot_chats(question: doc) -> dict:

    def question_prompt(s):
        return "Problem:" + "\n" + s["problem"]

    def answer_prompt(s):
        return  "Solution:" + "\n" + s["solution"]

    chats = []

    for qna in list_fewshot_samples():
        chats.append(
            {"role": "user", "content": question_prompt(qna)})
        chats.append(
            {"role": "assistant", "content": answer_prompt(qna)})

    # chats.append({"role": "user", "content": question_prompt(question)+
    #                                          " Let's think step by step. At the end, you MUST write the answer as an integer after '####'."})

    chats.append(
        {"role": "user", "content": question_prompt(question)})

    return chats

In [10]:
import requests

def get_llama2_response(messages):

    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3.2",
            "messages": messages,
            "stream": False
        }
    )

    response_data = response.json()
    return response_data['message']['content']

In [14]:
from datasets.math_dataset import process_results, get_unnormalized_answer
# for doc in tqdm.tqdm(category_data['algebra'][:10]):
#     prompt = prepare_sample(doc)
#


import os
if not os.path.exists('log'):
    os.makedirs('log')

log_file_path = 'log/errors_math.txt'
with open(log_file_path, 'w') as log_file:
    log_file.write('')


total = correct = 0
for qna in tqdm.tqdm(category_data['algebra'][:10]):

    messages = nshot_chats(qna)
    response = get_llama2_response(messages)


    correct_ans = normalize_final_answer(remove_boxed(last_boxed_only_string(qna["solution"])))
    doc = {'answer': correct_ans}

    res = process_results(doc, [response])['exact_match']


    unnormalized_answer = get_unnormalized_answer(response)
    answer = normalize_final_answer(unnormalized_answer)




    total += 1
    if not res:
        with open(log_file_path, 'a', encoding='utf-8') as log_file:
            log_file.write(f"{messages}\n\n")
            log_file.write(f"Response: {response}\n\n")
            log_file.write(f"Ground Truth: {doc['answer']}\n\n")
            log_file.write(f"Current Accuracy: {correct/total:.3f}\n\n")
            log_file.write('\n\n')
    else:
        correct += 1

print(f"Total Accuracy: {correct/total:.3f}")

 30%|███       | 3/10 [10:53<25:25, 217.98s/it]


KeyboardInterrupt: 